In [1]:
'''
! pip install langchain
! pip install openai
'''

'\n! pip install langchain\n! pip install openai\n'

In [2]:
import config
import os

os.environ['OPENAI_API_KEY'] = config.API_KEY

In [3]:
# from langchain.llms import OpenAI

# llm_opemAI = OpenAI(temperature=0.9)
# print(llm_opemAI.predict("What is the Capital of india?"))
# print(llm_opemAI("What would be a good company name for a company that makes colorful socks?"))

In [4]:
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.9)

In [5]:
text = "What would be a good company name for a company that makes colorful socks?"
print(llm(text))



Socktastic!


## Prompt 

In [6]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

In [7]:
prompt = PromptTemplate(
    input_variables=["Programming_language"],
    template="What are the top 5 Programming language youtube channel to learn {Programming_language}",
)

In [8]:
print(prompt.format(Programming_language="python"))
print(llm(prompt.format(Programming_language="python")))

What are the top 5 Programming language youtube channel to learn python


1. Corey Schafer
2. thenewboston
3. Derek Banas
4. Clever Programmer
5. Python Programmer


In [9]:
template = """
Interprete the text and evaluate the text.
Sentiment: Is the text is Neutral, Positive or negative
Subject: What is the subject text about? Use excatly one word

Format the output in proper JSON with following keys:
sentiment
subject
text: {input}"""

In [10]:
from langchain import PromptTemplate, OpenAI, LLMChain
from langchain.prompts import ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_template(template=template)
chain = LLMChain(llm=llm, prompt=prompt_template)
chain.predict(input="I purchase ChatGPT course, it was very much helpful for my learning.")


'\n\n{\n    "sentiment": "Positive",\n    "subject": "Learning",\n    "text": "I purchase ChatGPT course, it was very much helpful for my learning."\n}'

## USe of ResponseSchema, Template, Chain and OutputParser

In [11]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
sentiment_schema = ResponseSchema(name="sentiment", description="Is the text Neutral, Positive or negative ? Only provide single word answere")
subject_schema = ResponseSchema(name="subject", description="What subject is text about ? Use only single word answere")
price_schema = ResponseSchema(name="price", description="How expensive was the product ? Use 'price not availble' in the text if price is not present else give price in float")
response_scheme = [sentiment_schema, subject_schema, price_schema]


In [12]:
parser = StructuredOutputParser.from_response_schemas(response_scheme)
format_instruction = parser.get_format_instructions()
format_instruction

'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"sentiment": string  // Is the text Neutral, Positive or negative ? Only provide single word answere\n\t"subject": string  // What subject is text about ? Use only single word answere\n\t"price": string  // How expensive was the product ? Use \'price not availble\' in the text if price is not present else give price in float\n}\n```'

In [13]:
template = """
Interprete the text and evaluate the text.
Sentiment: Is the text is Neutral, Positive or negative
Subject: What is the subject text about? Use excatly one word

Format the output in proper JSON with following keys:
sentiment
subject
text: {input}
{format_instruction}"""

from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

prompt = ChatPromptTemplate.from_template(template=template)
format_instruction = parser.get_format_instructions()

message = prompt.format_messages(input="I purchase ChatGPT course in $ 299.98, it was very much helpful for my learning. ", format_instruction=format_instruction)
chat = ChatOpenAI(temperature=0.0)
response  = chat(message)
response

AIMessage(content='```json\n{\n\t"sentiment": "Positive",\n\t"subject": "Course",\n\t"price": 299.98\n}\n```\n\nInterpretation: The text is expressing a positive sentiment about a course that was purchased for $299.98. The subject of the text is the course.', additional_kwargs={}, example=False)

In [14]:
template = """
Interprete the text and evaluate the text.
Sentiment: Is the text is Neutral, Positive or negative
Subject: What is the subject text about? Use excatly one word

Return only JSON format output. Do not add any extara Interpretaion or text.
text: {input}
{format_instruction}"""

from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

prompt = ChatPromptTemplate.from_template(template=template)
format_instruction = parser.get_format_instructions()

message = prompt.format_messages(input="I purchase ChatGPT course in $ 299.98, it was very much helpful for my learning. ", format_instruction=format_instruction)
chat = ChatOpenAI(temperature=0.0)
response  = chat(message)
print(response)
response

content='```json\n{\n\t"sentiment": "Positive",\n\t"subject": "Course",\n\t"price": 299.98\n}\n```' additional_kwargs={} example=False


AIMessage(content='```json\n{\n\t"sentiment": "Positive",\n\t"subject": "Course",\n\t"price": 299.98\n}\n```', additional_kwargs={}, example=False)

In [15]:
output_dict = parser.parse(response.content)
output_dict

{'sentiment': 'Positive', 'subject': 'Course', 'price': 299.98}

## Chaining


**Definition**: Chains are one of the fundamental building blocks of this library LangChain.

The official definition of chains is as follows:

> A chain is composed of links, which can be either primitives or other chains. Primitives can be prompts, LLMs, utilities, or other chains.

So, a chain essentially functions as a pipeline that processes an input using a specific combination of primitives. In simpler terms, it can be seen as a 'step' that performs a defined set of operations on an input and produces a result. Chains can range from a prompt-based pass through a large language model (LLM) to applying a Python function to a text.

Chains are categorized into three types: Utility chains, Generic chains, and Combine Documents chains. For this discussion, we will focus on the first two, as the third type is more specialized and will be covered later.

1. Utility Chains: These chains are commonly used to extract specific answers from an LLM with a narrow purpose. They are readily available for immediate use.
2. Generic Chains: These chains serve as building blocks for other chains but cannot be used independently.


In [30]:
# from langchain.chains import LLMChain, MultiPromptChain
from langchain.chains import LLMChain, LLMMathChain, TransformChain, SequentialChain
from langchain import PromptTemplate, OpenAI
from langchain.callbacks import get_openai_callback
import inspect
from langchain.prompts import ChatPromptTemplate

In [22]:
template = """
Interprete the text and evaluate the text.
Sentiment: Is the text is Neutral, Positive or negative
Subject: What is the subject text about? Use excatly one word

Format the output in proper JSON with following keys:
sentiment
subject
text: {input}
"""

llm = OpenAI(temperature=0.0)
prompt_template = PromptTemplate.from_template(template=template)
chain = LLMChain(llm=llm, prompt=prompt_template)
print(chain.predict(input ="Java is use in most of complex web applications"))


{
    "sentiment": "Neutral",
    "subject": "Java",
    "text": "Java is use in most of complex web applications"
}


In [26]:
#  verbose=True to see what the different steps in the chain are!
llm_math = LLMMathChain(llm=llm, verbose=True)

def count_tokens(chain, query):
    with get_openai_callback() as cb:
        result = chain.run(query)
        print(f'Spent a total of {cb.total_tokens} tokens')

    return result

count_tokens(llm_math, "What is 13 raised to the .3432 power?")



> Entering new  chain...
What is 13 raised to the .3432 power?


```text
13**.3432
```
...numexpr.evaluate("13**.3432")...

Answer: 2.4116004626599237
> Finished chain.
Spent a total of 264 tokens


'Answer: 2.4116004626599237'

In [27]:
# The question we send as input to the chain is not the only input that the LLM receives. The input is inserted into a broader context, which provides specific instructions on how to interpret the input we send. This broader context is known as a _prompt_. Now, let's examine what the prompt for this chain is!
print(llm_math.prompt.template)

Translate a math problem into a expression that can be executed using Python's numexpr library. Use the output of running this code to answer the question.

Question: ${{Question with math problem.}}
```text
${{single line mathematical expression that solves the problem}}
```
...numexpr.evaluate(text)...
```output
${{Output of running the code}}
```
Answer: ${{Answer}}

Begin.

Question: What is 37593 * 67?
```text
37593 * 67
```
...numexpr.evaluate("37593 * 67")...
```output
2518731
```
Answer: 2518731

Question: 37593^(1/5)
```text
37593**(1/5)
```
...numexpr.evaluate("37593**(1/5)")...
```output
8.222831614237718
```
Answer: 8.222831614237718

Question: {question}



In [28]:
# we set the prompt to only have the question we ask
prompt = PromptTemplate(input_variables=['question'], template='{question}')
llm_chain = LLMChain(prompt=prompt, llm=llm)

# we ask the llm for the answer with no context

count_tokens(llm_chain, "What is 13 raised to the .3432 power?")

Spent a total of 17 tokens


'\n\n2.907'

Above answere is wrong

Herein lies the power of prompting and one of our most significant insights thus far:

Insight: By utilizing prompts strategically, we can guide the behavior of the LLM, explicitly programming it to avoid common pitfalls and behave in a specific manner.

In [29]:
13**(.3432) ## Here is teh correct answere

2.4116004626599237

In [32]:
# The chain not only processes an input through the LLM but also incorporates Python code compilation at a later stage.
print(inspect.getsource(llm_math._call))

    def _call(
        self,
        inputs: Dict[str, str],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        _run_manager = run_manager or CallbackManagerForChainRun.get_noop_manager()
        _run_manager.on_text(inputs[self.input_key])
        llm_output = self.llm_chain.predict(
            question=inputs[self.input_key],
            stop=["```output"],
            callbacks=_run_manager.get_child(),
        )
        return self._process_llm_result(llm_output, _run_manager)



In [33]:
import re
def transform_func(inputs: dict) -> dict:
    text = inputs["text"]
    
    # replace multiple new lines and multiple spaces with a single one
    text = re.sub(r'(\r\n|\r|\n){2,}', r'\n', text)
    text = re.sub(r'[ \t]+', ' ', text)

    return {"output_text": text}

In [34]:
clean_extra_spaces_chain = TransformChain(input_variables=["text"], output_variables=["output_text"], transform=transform_func)
clean_extra_spaces_chain.run('A random text  with   some irregular spacing.\n\n\n     Another one   here as well.')

'A random text with some irregular spacing.\n Another one here as well.'

In [36]:
template = """Paraphrase this text:

{output_text}

In the style of a {style}.

Paraphrase: """
prompt = PromptTemplate(input_variables=["style", "output_text"], template=template)


style_paraphrase_chain = LLMChain(llm=llm, prompt=prompt, output_key='final_output')

In [37]:
sequential_chain = SequentialChain(chains=[clean_extra_spaces_chain, style_paraphrase_chain], input_variables=['text', 'style'], output_variables=['final_output'])

In [38]:
input_text = """Chains provide a powerful mechanism for integrating various components into a unified application.

For instance, we can construct a chain that receives user input, 
applies a formatting PromptTemplate to it,  

and subsequently feeds the formatted response into an LLM. Complex chains can be built by combining multiple chains 

or by incorporating chains with other components.
"""

In [39]:
count_tokens(sequential_chain, {'text': input_text, 'style': 'a 90s rapper'})

Spent a total of 168 tokens


"\nChains be like a strong tool for puttin' together different pieces to make one big app. Ya know? Like, we can take user input, style it up with a PromptTemplate, then feed it to an LLM. We can even get more complex by mixin' multiple chains or addin' other components. Word."

In [40]:
count_tokens(sequential_chain, {'text': input_text, 'style': 'python professor'})

Spent a total of 163 tokens


'\nChains are a great way to combine different components into a single application. For example, we can create a chain that takes user input, formats it with a PromptTemplate, and then sends the formatted output to an LLM. We can also create more complex chains by combining multiple chains or by adding chains to other components.'

##  langchain-hub

Loading a chain from the Langchain hub is a straightforward process. It involves locating the desired chain within the repository and utilizing the `load_chain` function with the appropriate path. Additionally, there are other functions available such as `load_prompt` and `initialize_agent`, which we will discuss in detail at a later stage. Now, let's explore how we can accomplish this with our previously mentioned LLMMathChain.

In [41]:
from langchain.chains import load_chain

In [44]:
llm_math_chain = load_chain('lc://chains/llm-math/chain.json')

In [45]:
llm_math_chain.verbose

True

In [46]:
llm_math_chain.verbose = False
llm_math_chain.verbose

False